# Introduction to Penguin Data Analysis

Welcome to this analysis of penguin data using Python and machine learning. The goal is to apply K-Means clustering to explore how penguin species are grouped based on physical and ecological traits.

Analysis Pipeline
Our approach includes:

* Data Importation and Cleaning: Preparing the dataset by addressing missing values and errors.
* Exploratory Data Analysis (EDA): Visualizing data to understand distributions and relationships.
* Feature Selection and Preprocessing: Selecting significant features and normalizing data.
* Clustering: Using K-Means to form groups and analyzing these clusters.
* Evaluation: Comparing clustering results.


The 'Species' column, which shows the actual species, will be set aside until after we finish clustering. This way, we can compare our results with the true species to check the accuracy of our model. We will evaluate the performance by using multiple scoring metrics such as silhouette scores, Davies-Bouldin index, and the Calinski-Harabasz index as well.


______________________________

# Packages

In [ ]:
# Data Manipulation and Handling
import numpy as np
import pandas as pd

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Statistical Analysis
import scipy.stats as stats

# Preprocessing Techniques
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

# Machine Learning - Dimensionality Reduction
from sklearn.decomposition import PCA

# Machine Learning - Clustering
from sklearn.cluster import KMeans

# Machine Learning - Evaluation Metrics
from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score
from sklearn.metrics import davies_bouldin_score
from sklearn.metrics import accuracy_score

# Machine Learning - Visualization
from yellowbrick.cluster import KElbowVisualizer

# Warnings
import warnings

# Raw Data

In [ ]:
# Data import
data_original = pd.read_csv("/kaggle/input/palmer-archipelago-antarctica-penguin-data/penguins_lter.csv")
data_original.head(3)

In [ ]:
#Summary Function
pd.options.display.float_format = '{:,.2f}'.format
def summary(df):
    print(f'data shape: {df.shape}')
    summ = pd.DataFrame(df.dtypes, columns=['data type'])
    summ['#missing'] = df.isnull().sum().values
    summ['%missing'] = df.isnull().sum().values / len(df) * 100
    summ['#unique'] = df.nunique().values
    desc = pd.DataFrame(df.describe(include='all').transpose())
    summ['min'] = desc['min'].values
    summ['max'] = desc['max'].values
    summ['average'] = desc['mean'].values
    summ['standard_deviation'] = desc['std'].values
    return summ

print('\033[93m\033[1m' + 'Summary Table: ' + '\033[0m')
summary(data_original)

In [ ]:
# Highlighting null values
is_null = data_original.isnull()

plt.figure(figsize=(10, 8))
sns.heatmap(is_null, cbar=False, cmap='viridis', yticklabels=False)

plt.show()

We will be dropping rows with null values in Culmen Length, Culmen Depth, Flipper Length, and Body Mass as they constitute only 0.58% of the data points.


We will impute the null values in the Sex, Delta 15 N, and Delta 13 C columns. The mode will be used to impute values for Sex, while Delta 15 N and Delta 13 C will be imputed with means or medians based on whether their data distributions are normal or skewed.

The Sex column has three unique values, indicating a possible error in data entry, since there are typically only two sexes.

No significant outliers were observed considering the min, max, average, and std values.

# Data Cleaning

**Chosen Columns for Clustering:**

**Species:** Chosen to validate our clustering performance; the true species labels can help assess how well the clustering algorithm has grouped similar observations together.

**Island:** Selected because different islands might host slightly different ecological environments, influencing the penguins' physical traits and behaviors, which could affect clustering.

**Culmen Length (mm):** Important as it is a key morphological trait that influences feeding strategy and habitat utilization, relevant for distinguishing clusters.

**Culmen Depth (mm):** Included as it provides insight into the feeding mechanics and diet preferences of the penguins, which are critical for differentiating species and potentially clustering them by ecological niches.

**Flipper Length (mm):** Essential for clustering as it correlates with the penguin's swimming ability and migration behavior, factors that vary among species and could define cluster boundaries.

**Body Mass (g):** Penguins with different body masses may show variations in migratory behavior, reproductive success, and foraging patterns, which can naturally cluster based on these traits.

**Sex:** Included as male and female penguins often show distinct size dimorphism and could form naturally segregating clusters within species.

**Delta 15 N (o/oo):** Chosen because it indicates the trophic level at which the penguins feed, providing insights into their ecological role that may influence how they cluster by diet type.

**Delta 13 C (o/oo):** Selected to give clues about the dietary sources of the penguins (inshore vs. offshore), which is a vital ecological factor that could impact clustering by showing different feeding grounds or behaviors.

In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)

#Chosen columns for analysis
important_columns = ['Species',
                     'Island',
                     'Culmen Length (mm)',
                     'Culmen Depth (mm)',
                     'Flipper Length (mm)',
                     'Body Mass (g)',
                     'Sex',
                     'Delta 15 N (o/oo)',
                     'Delta 13 C (o/oo)']

data = data_original[important_columns].copy()

In [ ]:
summary(data)

In [ ]:
# Sex & Delta columns distributions.

plt.figure(figsize=(20,3))

plt.subplot(1,3,1)
sns.histplot(data=data, x='Delta 15 N (o/oo)', color='#5A9')

plt.subplot(1,3,2)
sns.histplot(data=data, x='Delta 13 C (o/oo)', color='#B6D7A8')

plt.subplot(1,3,3)
sex_counts = data['Sex'].value_counts().reset_index()
sex_counts.columns = ['Sex', 'Count']
sns.barplot(data=sex_counts, x='Sex', y='Count', color="#A2C4C9")

plt.show()

data['Sex'].value_counts()

The distributions of the Delta 15 N and Delta 13 C variables are right-skewed, so the median will be used to impute the missing values.

In the 'Sex' column, the third unique value is '.', which occurs only once.
To achieve a roughly equal distribution between male and female categories, the dot and any missing values in the 'Sex' column will be replaced with 'FEMALE,' aiming for a closest 50-50 male-to-female ratio.

In [ ]:
# Data imputation for Delta columns
delta_15n_median = data['Delta 15 N (o/oo)'].median()
delta_13c_median = data['Delta 13 C (o/oo)'].median()

# Use .loc[] to ensure the operation affects the original DataFrame
data.loc[data['Delta 15 N (o/oo)'].isna(), 'Delta 15 N (o/oo)'] = delta_15n_median
data.loc[data['Delta 13 C (o/oo)'].isna(), 'Delta 13 C (o/oo)'] = delta_13c_median

# Data imputation for Sex
data.loc[data['Sex'] == '.', 'Sex'] = 'FEMALE'
data.loc[data['Sex'].isna(), 'Sex'] = 'FEMALE'

# Dropping any remaining rows with null values
data.dropna(inplace=True)

In [ ]:
summary(data)

# Exploratory Data Analysis

In [ ]:
# Color palette
color_palette = sns.color_palette(["#D5A6BD", "#93C47D", "#F9CB9C",
                                   "#8E7CC3", "#C27BA0", "#77A1D3",
                                   "#B4A7D6", "#A2C4C9","#FFE599",
                                   ])

sns.palplot(color_palette)
plt.show()

black_palette = ['#100C07', '#3E3B39', '#6D6A6A', '#9B9A9C', '#CAC9CD']

sns.palplot(black_palette)
plt.show()

In [ ]:

# Plotting Num Features
numerical_column_names = data.select_dtypes('number').columns.tolist()

for color, column in enumerate(numerical_column_names):
  print(f'\033[93m\033[1m Data Distribution : {column}\033[0m')
  plt.figure(figsize=(14, 2))
  plt.subplot(1, 2, 1)
  plt.title(f'{column} Histogram', fontweight = 'bold', fontsize = 14, fontfamily = 'sans-serif', color = black_palette[2])
  sns.histplot(data[column], kde=True, color = color_palette[color])
  plt.subplot(1, 2, 2)
  plt.title(f'{column} Boxplot', fontweight = 'bold', fontsize = 14, fontfamily = 'sans-serif', color = black_palette[2])
  sns.boxenplot(data=data, x=column, color = color_palette[color])
  plt.show()
  print('\n')

# Plotting Object Features
object_column_names = data.select_dtypes('object').columns.tolist()
object_column_names.remove('Species')

for column in object_column_names:
      print(f'\033[93m\033[1m Pie Chart : {column}\033[0m')
      plt.figure(figsize=(10, 6))
      data[column].value_counts().plot(kind='pie', colors=color_palette, autopct='%1.1f%%', wedgeprops={'edgecolor': black_palette[2]})
      plt.title(f'{column} Pie Chart', fontweight = 'bold', fontsize = 14, fontfamily = 'sans-serif', color = black_palette[2])
      plt.ylabel('')
      plt.show()
      print('\n')

In [ ]:
#Correlation heatmap
plt.figure(figsize=(10, 6))
plt.title('Correlation Heatmap', fontweight = 'bold', fontsize = 14, fontfamily = 'sans-serif', color = black_palette[2])
correlation_matrix = data[numerical_column_names].corr()
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, annot=True, mask = mask, cmap='coolwarm', center=0)
plt.show()

In [ ]:
#Pairplot on Sex as hue
pair_plot = sns.pairplot(data, hue='Sex', markers=['o', '*'], diag_kind='kde', palette='husl', corner='lower')
plt.show()

In [ ]:
#Pairplot on Island as hue
pair_plot = sns.pairplot(data, hue='Island', markers=['o', '^', '*'], diag_kind='kde', palette='husl', corner='lower')
plt.show()

# Clustering

In [ ]:
# Filtering Species out
clustering_data = data.iloc[:, 1:]
clustering_data.sample(3)

In [ ]:
# One-Hot Encoding for object columns
encoder = OneHotEncoder()
encoded_data = encoder.fit_transform(clustering_data[object_column_names])

# Standard Scaling for numerical columns
scaler = StandardScaler()
scaled_data = scaler.fit_transform(clustering_data[numerical_column_names])

# Create a dataframe of the final result
prepared_data = pd.concat([pd.DataFrame(encoded_data.toarray()), pd.DataFrame(scaled_data)], axis=1)

encoded_column_names = encoder.get_feature_names_out(input_features=object_column_names)
all_column_names = list(encoded_column_names) + numerical_column_names
prepared_data.columns = all_column_names

prepared_data = pd.DataFrame(prepared_data, columns=all_column_names)

prepared_data.sample(3)

In [ ]:
# Cumulative variance explained by each PCA component.
pca = PCA()
pca.fit(prepared_data)

cumulative_variance = pca.explained_variance_ratio_.cumsum()

# Plot
plt.figure(figsize=(8, 4))
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o')
plt.title('Cumulative Variance Explained by PCA Components')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.grid(True)
plt.show()

In [ ]:
# Choose the number of components that explain at least 90% of the variance
n_components = sum(cumulative_variance < 0.90) + 1

pca = PCA(n_components=n_components)
pca_data = pca.fit_transform(prepared_data)

# Create the pca_data
pca_data = pd.DataFrame(data=pca_data, columns=[f'PC{i}' for i in range(1, n_components + 1)])

print('\n')
print(f'Chosen Number of Components is:  {n_components}')

print('\n')
pca_data.head(3)

**KElbowVisualizer (Elbow Method):**

* Metric Used: Often uses the within-cluster sum of squares (inertia), which measures the compactness of clusters. The elbow method identifies the number of clusters at which the decrease in inertia becomes less pronounced (the "elbow point").
* Interpretation: It tends to favor more clusters because it primarily considers how tightly grouped the data points are within each cluster.


In [ ]:
print('Elbow Method to determine the number of clusters to be formed:')
Elbow_M = KElbowVisualizer(KMeans(n_init=10), k=10)
Elbow_M.fit(pca_data)
Elbow_M.show()

In [ ]:
# Clustering PCA data
kmeans = KMeans(n_clusters=5, n_init=10, random_state=42)
kmeans.fit(pca_data)
labels = kmeans.labels_
data['Cluster_PCA'] = labels

# Performance Evaluation


In [ ]:
# Pairplot of predicted clusters
pair_plot = sns.pairplot(data, hue='Cluster_PCA', diag_kind='kde', palette=sns.color_palette('husl', 5), corner='lower')
plt.show()

We can observe that the brown and purple segments typically move together, and one usually indicates lower physical properties in penguins. This relationship is very similar to what we saw in our exploratory data analysis (EDA), particularly in the pairplot visualization of the data points with the 'Sex' column as the hue. A similar relationship can also be observed between the blue and red data points.

This suggests that our algorithm has successfully captured the differences between each cluster based on their sex.

However, in this case, since we have 5 clusters and there are 2 sexes, we would expect the number of clusters to be divisible by 2. Let's first calculate the performance metrics, and then increase our cluster size to 6 to see how the performance metrics change.

____________________________

**Silhouette Score:**

* Metric Used: Measures the mean intra-cluster distance (a) and the mean nearest-cluster distance (b) for each sample.
* Interpretation: A higher silhouette score indicates better-defined clusters. It balances both the compactness and the separation between clusters.

**Davies-Bouldin Score:**

* Metric Used: Evaluates the average similarity between each cluster with its most similar cluster, where similarity is the ratio of within-cluster distances to between-cluster distances.
* Interpretation: Lower values indicate better clustering, suggesting that clusters are farther apart and less dispersed.

**Calinski-Harabasz Score:**

* Metric Used: Based on the ratio of the sum of between-clusters dispersion to within-cluster dispersion for all clusters (where higher values generally indicate clusters are well-separated and dense).
* Interpretation: Higher values indicate better-defined clusters.

In [ ]:
# Calculating the performance scores on 5 segments
silhouette_scaled = silhouette_score(pca_data, data['Cluster_PCA'])
calinski_scaled = calinski_harabasz_score(pca_data, data['Cluster_PCA'])
davies_scaled = davies_bouldin_score(pca_data, data['Cluster_PCA'])

print("Silhouette Score:", silhouette_scaled)
print("Calinski-Harabasz Score:", calinski_scaled)
print("Davies-Bouldin Score:", davies_scaled)

Increasing cluster size

- Let's increase the cluster size to 6.

In [ ]:
# Clustering PCA data
kmeans = KMeans(n_clusters=6, n_init=10, random_state=42)
kmeans.fit(pca_data)
labels = kmeans.labels_
data['Cluster_PCA'] = labels

In [ ]:
# Pairplot of predicted 6 clusters
pair_plot = sns.pairplot(data, hue='Cluster_PCA', diag_kind='kde', palette=sns.color_palette('husl', 6), corner='lower')
plt.show()

In [ ]:
# Calculating the scores of 6 clusters
silhouette_scaled = silhouette_score(pca_data, data['Cluster_PCA'])
calinski_scaled = calinski_harabasz_score(pca_data, data['Cluster_PCA'])
davies_scaled = davies_bouldin_score(pca_data, data['Cluster_PCA'])

print("Silhouette Score:", silhouette_scaled)
print("Calinski-Harabasz Score:", calinski_scaled)
print("Davies-Bouldin Score:", davies_scaled)

Increasing our cluster size to six resulted in only a slight improvement in our performance scores. However, we actually have a 'species' column available in our dataset. We know that our algorithm already captures the effect of 'Sex' on the data features. Let's create a new column by combining 'Sex' and 'Species' and then map these values to the model predictions to calculate an accuracy score in order to see how well our model actually performed.

In [ ]:
# Combining Sex and Species
data['Sex_Species'] = data['Sex'] + '_' + data['Species']
data['Sex_Species'].value_counts()

In [ ]:
# Mapping Values
mapping = data[['Cluster_PCA', 'Sex_Species']].dropna().groupby('Cluster_PCA')['Sex_Species'].agg(lambda x: x.value_counts().idxmax())
data['Predicted_Species'] = data['Cluster_PCA'].map(mapping)

# Calculate the accuracy
accuracy = accuracy_score(data['Sex_Species'], data['Predicted_Species'])
print("Accuracy of k-means clustering:", accuracy)

Our model achieves an accuracy of 92% on the combined 'Sex' and 'Species' categories. The "accuracy" here measures the correspondence between cluster assignments and pre-labeled categories. It's more about how well the clusters represent known categories rather than a traditional accuracy measure.